In [18]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import nltk

tokenizer = AutoTokenizer.from_pretrained("Newtral/xlm-r-finetuned-toxic-political-tweets-es")

In [19]:
tokens = tokenizer("Buenas tardes, como están ustedes?")

In [20]:
# https://huggingface.co/Newtral/xlm-r-finetuned-toxic-political-tweets-es

{'input_ids': [0, 161457, 7, 15234, 7, 4, 533, 6805, 172540, 32, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}